# OpenAI API로 함수 호출하기

In [1]:
import os
import openai
import json
from dotenv import load_dotenv

load_dotenv()

# set openai api key
openai.api_key = os.environ['OPENAI_API_KEY']

### 가져오기 완료 함수 정의하기

In [2]:
def get_completion(messages, model="gpt-3.5-turbo-1106", temperature=0, max_tokens=300, tools=None, tool_choice=None):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        tools=tools,
        tool_choice=tool_choice
    )
    return response.choices[0].message

### 더미 함수 정의

In [3]:
# Defines a dummy function to get the current weather
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather = {
        "location": location,
        "temperature": "50",
        "unit": unit,
    }

    return json.dumps(weather)

### 함수 정의

OpenAI 문서에 설명된 대로, 다음은 요청에 포함될 함수를 정의하는 간단한 예시입니다.

설명은 LLM에 직접 전달되고 LLM은 설명을 사용하여 함수를 사용할지 여부 또는 사용/호출 방법을 결정하기 때문에 설명이 중요합니다.

In [4]:
# define a function as tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },   
    }
]

In [5]:
# define a list of messages

messages = [
    {
        "role": "user",
        "content": "What is the weather like in London?"
    }
]

In [6]:
response = get_completion(messages, tools=tools)
print(response)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GYg3yhSh1bbMabne9brwTqmA', function=Function(arguments='{"location":"London","unit":"celsius"}', name='get_current_weather'), type='function')])


응답.도구_통화[0].함수.인수

이제 인수를 캡처할 수 있습니다:

In [7]:
args = json.loads(response.tool_calls[0].function.arguments)

In [8]:
get_current_weather(**args)

'{"location": "London", "temperature": "50", "unit": "celsius"}'

### 함수 호출 동작 제어하기

LLM 기반 대화형 에이전트의 컨텍스트에서 이 'function_calling' 기능을 설계하는 데 관심이 있다고 가정해 봅시다. 그러면 솔루션에서 어떤 함수를 호출할지 또는 아예 호출해야 하는지 알아야 합니다. 인사말 메시지의 간단한 예를 들어 보겠습니다:

In [9]:
messages = [
    {
        "role": "user",
        "content": "Hello! How are you?",
    }
]

In [10]:
get_completion(messages, tools=tools)

ChatCompletionMessage(content="Hello! I'm here and ready to assist you. How can I help you today?", role='assistant', function_call=None, tool_calls=None)

함수 호출에서 원하는 동작을 지정하여 시스템의 동작을 제어할 수 있습니다. 기본적으로 모델은 함수를 호출할지 여부와 호출할 함수를 스스로 결정합니다. 이는 기본 설정인 `tool_choice: "auto"`를 설정하면 됩니다.  

In [11]:
get_completion(messages, tools=tools, tool_choice="auto")

ChatCompletionMessage(content="Hello! I'm here and ready to assist you. How can I help you today?", role='assistant', function_call=None, tool_calls=None)

tool_choice: "none"`을 설정하면 모델이 제공된 기능을 사용하지 않도록 강제합니다.

In [12]:
get_completion(messages, tools=tools, tool_choice="none")

ChatCompletionMessage(content="Hello! I'm here and ready to assist you. How can I help you today?", role='assistant', function_call=None, tool_calls=None)

In [13]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in London?",
    }
]
get_completion(messages, tools=tools, tool_choice="none")

ChatCompletionMessage(content='I will check the current weather in London for you.', role='assistant', function_call=None, tool_calls=None)

애플리케이션에서 원하는 동작이 있는 경우 모델이 함수를 선택하도록 강제할 수도 있습니다. 예시:

In [14]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in London?",
    }
]
get_completion(messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_current_weather"}})

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ctUCZtwxZYhfF3sirByNY2qC', function=Function(arguments='{"location":"London","unit":"celsius"}', name='get_current_weather'), type='function')])

OpenAI API는 여러 함수를 한 번에 호출할 수 있는 병렬 함수 호출도 지원합니다.

In [15]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in London and Belmopan in the coming days?",
    }
]
get_completion(messages, tools=tools)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GhyrfrY9QWkrJs9vKqxpGW22', function=Function(arguments='{"location": "London", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_Cl9fqGh1AZgi4yD8n5Lrc6NE', function=Function(arguments='{"location": "Belmopan", "unit": "celsius"}', name='get_current_weather'), type='function')])

위의 응답에서 쿼리한 두 위치에 대한 함수 호출 정보가 응답에 포함되어 있음을 확인할 수 있습니다.

모델 피드백을 위한 ### 함수 호출 응답

함수 호출에서 생성된 입력으로 API를 호출한 후 얻은 결과를 다시 전달하는 에이전트를 개발하는 데 관심이 있을 수도 있습니다. 다음 예제를 살펴보겠습니다:


In [16]:
messages = []
messages.append({"role": "user", "content": "What's the weather like in Boston!"})
assistant_message = get_completion(messages, tools=tools, tool_choice="auto")
assistant_message = json.loads(assistant_message.model_dump_json())
assistant_message["content"] = str(assistant_message["tool_calls"][0]["function"])

#a temporary patch but this should be handled differently
# remove "function_call" from assistant message
del assistant_message["function_call"]

In [17]:
messages.append(assistant_message)

그런 다음 `get_current_weather` 함수의 결과를 추가하고 `tool` 역할을 사용하여 모델에 다시 전달합니다.

In [18]:
# get the weather information to pass back to the model
weather = get_current_weather(messages[1]["tool_calls"][0]["function"]["arguments"])

messages.append({"role": "tool",
                 "tool_call_id": assistant_message["tool_calls"][0]["id"],
                 "name": assistant_message["tool_calls"][0]["function"]["name"],
                 "content": weather})

In [19]:
final_response = get_completion(messages, tools=tools)

In [20]:
final_response

ChatCompletionMessage(content='The current temperature in Boston, MA is 50°F.', role='assistant', function_call=None, tool_calls=None)